In [ ]:
# Cell 1: Core Components with Fixed Scaling
import open3d as o3d
import numpy as np

DEFAULT_PARAMS = {
    'body_scale': [1.8, 1.2, 0.7],  # Wider chest, shorter height
    'head_scale': [0.9, 0.7, 0.6],   # Squarer head shape
    'leg_radius': 0.2,               # Thicker legs
    'jaw_depth': 0.4                 # New parameter for bulldog jaw
}

def create_part(shape, params, position=(0,0,0), rotation=None):
    mesh = getattr(o3d.geometry.TriangleMesh, f'create_{shape}')(**params)
    mesh.compute_vertex_normals()
    
    # Create transformation matrix
    transform = np.eye(4)
    transform[:3, 3] = position  # Set translation
    
    if rotation:
        # Convert rotation angles to radians and create rotation matrix
        theta_x, theta_y, theta_z = np.radians(rotation)
        # ... (rotation matrix creation would go here)
    
    mesh.transform(transform)
    return mesh

# Cell 2: Enhanced Bulldog Generator
class BulldogGenerator:
    def __init__(self):
        self.components = []
        self.params = DEFAULT_PARAMS
        
    def _apply_scaling(self, mesh, scale_factors):
        """Proper non-uniform scaling using transformation matrix"""
        scaling_matrix = np.eye(4)
        scaling_matrix[0, 0] = scale_factors[0]  # X-axis
        scaling_matrix[1, 1] = scale_factors[1]  # Y-axis
        scaling_matrix[2, 2] = scale_factors[2]  # Z-axis
        mesh.transform(scaling_matrix)
        return mesh
    
    def _create_head(self):
        """Complex bulldog head with jaw structure"""
        # Main head box
        head = create_part('box', {'width': 1, 'height': 1, 'depth': 1})
        self._apply_scaling(head, self.params['head_scale'])
        head.translate((self.params['body_scale'][0]/2, 0, 0.4))
        
        # Underbite jaw
        jaw = create_part('box', {'width': 0.8, 'height': 0.4, 'depth': 0.3},
                        position=(self.params['body_scale'][0]/2 + 0.2, 0, 0.3))
        
        # Combine head parts
        self.components.extend([head, jaw])
    
    def _create_body(self):
        """Muscular bulldog torso"""
        body = create_part('sphere', {'radius': 1})
        self._apply_scaling(body, self.params['body_scale'])
        self.components.append(body)
    
    def _create_legs(self):
        """Thick, short bulldog legs"""
        leg_params = {
            'radius': self.params['leg_radius'],
            'height': self.params['body_scale'][2] * 0.8
        }
        
        positions = [
            (0.6, 0.6, -self.params['body_scale'][2]/2),
            (-0.6, 0.6, -self.params['body_scale'][2]/2),
            (0.6, -0.6, -self.params['body_scale'][2]/2),
            (-0.6, -0.6, -self.params['body_scale'][2]/2)
        ]
        
        for pos in positions:
            leg = create_part('cylinder', leg_params, position=pos)
            self.components.append(leg)
    
    def generate(self):
        self._create_body()
        self._create_head()
        self._create_legs()
        
        # Merge components
        bulldog = self.components[0]
        for part in self.components[1:]:
            bulldog += part
            
        return bulldog

# Cell 3: Generation & Visualization
generator = BulldogGenerator()
bulldog = generator.generate()

print("Bulldog components:", [type(m).__name__ for m in generator.components])
o3d.visualization.draw_geometries([bulldog],
                                window_name="Enhanced Bulldog",
                                width=800,
                                height=600)

TypeError: scale(): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.cpu.pybind.geometry.Geometry3D, scale: float, center: numpy.ndarray[numpy.float64[3, 1]]) -> open3d.cpu.pybind.geometry.Geometry3D
    2. (self: open3d.cpu.pybind.geometry.Geometry3D, scale: float, center: numpy.ndarray[numpy.float64[3, 1]]) -> open3d.cpu.pybind.geometry.Geometry3D

Invoked with: TriangleMesh with 762 points and 1520 triangles., [1.5, 1.0, 0.6]; kwargs: center=array([-1.52619241e-17,  1.78571797e-17,  2.18547839e-17])